In [1]:
import logging
import pandas as pd
from sklearn.model_selection import train_test_split
import os.path
from os import path
import pandas as pd
import numpy as np
import base64
import re

In [2]:
from transformers import TFBertModel,  BertConfig, BertTokenizerFast

# Then what you need from tensorflow.keras
from tensorflow.keras.layers import Input, Dropout, Dense
from tensorflow.keras.models import Model, save_model, load_model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.initializers import TruncatedNormal
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.metrics import CategoricalAccuracy
from tensorflow.keras.utils import to_categorical

In [3]:
import logging
logging.basicConfig(format='%(asctime)s - %(levelname)s - %(name)s -   %(message)s',
                    datefmt='%m/%d/%Y %H:%M:%S',
                    level=logging.INFO)
logger = logging.getLogger(__name__)

In [4]:
poem_with_tag_df = pd.read_csv('poems_with_new_tags.zip', compression='zip', header=0, quotechar='"')
poem_without_tag_df = pd.read_csv('poems_without_tags.zip', compression='zip', header=0, quotechar='"')


In [5]:
poem_with_tag_df.head()

,content,dynasty,author,tags,star,author_stars,title,tags_list,new_tags,new_first_tag,写物,劝勉,家庭,快乐,悲苦,政治,朋友,游玩
0,《吴都赋》云：“户藏烟浦，家具画船。”唯吴兴为然。春游之盛，西湖未能过也。己酉岁，予与萧时父...,宋代,姜夔,春游;怀人;,64,279,琵琶仙·《吴都赋》云：「户藏烟浦,"['春游', '怀人']",政治;游玩,政治,0,0,0,0,0,1,0,1
1,《廿一史弹词》第三段说秦汉开场词滚滚长江东逝水，浪花淘尽英雄。是非成败转头空。青山依旧在，几...,明代,杨慎,咏史;抒怀;人生;哲理,3244,131,临江仙·滚滚长江东逝水,"['咏史', '抒怀', '人生', '哲理']",写物;劝勉;悲苦;政治,写物,1,1,0,0,1,1,0,0
2,《水经》云：“彭蠡之口有石钟山焉。”郦元以为下临深潭，微风鼓浪，水石相搏，声如洪钟。是说也，...,宋代,苏轼,古文观止;纪游;写景;写山,306,4011,石钟山记,"['古文观止', '纪游', '写景', '写山']",写物;政治;游玩,写物,1,0,0,0,0,1,0,1
3,【序】辛亥之冬，予载雪诣石湖。止既月，授简索句，且征新声，作此两曲。石湖把玩不已，使工妓隶习...,宋代,姜夔,咏物;梅花,74,279,暗香疏影,"['咏物', '梅花']",劝勉,劝勉,0,1,0,0,0,0,0,0
4,〔一枝花〕 攀出墙朵朵花，折临路枝枝柳。花攀红蕊嫩，柳折翠条柔，浪子风流。凭着我折柳攀花手...,元代,关汉卿,;散曲;抒情;生活,140,124,【南吕】一枝花不伏老,"['散曲', '抒情', '生活']",写物;快乐;悲苦,写物,1,0,0,1,1,0,0,0


In [6]:
poem_without_tag_df.head()

,content,dynasty,author,tags,star,author_stars,title,tags_list
0,一上一上又一上，一上直到高山上。举头红日白云低，四海五湖皆一望。,明代,唐寅,;,1,231,伯虎扮乞儿，作诗骗酒,['']
1,一个兰舟，双桂桨、顺流东去。但满目、银光万顷，凄其风露。渔火已归鸿雁汊，棹歌更在鸳鸯浦。渐夜...,宋代,汪元量,;,2,57,满江红（吴江秋夜）,['']
2,一从簪笏事金貂，每借温颜放折腰。长觉身轻离泥滓，忽惊手重捧琼瑶。马因回顾虽增价，桐遇知音已半...,唐代,白居易,;,1,5357,江西裴常侍以优礼见待又蒙赠诗辄叙鄙诚用伸感谢,['']
3,一以我为牛，一以吾为马。人与之名受不辞，善学庄周者。江海任虚舟，风雨从飘瓦。醉者乘车坠不伤，...,宋代,辛弃疾,;,3,2593,卜算子（用庄语）,['']
4,一分素景，千家新月，凉露楼台遍洗。宝奁深夜结蛛丝，纴五孔、金针不寐。,宋代,石延年,;,1,6,鹊桥仙·一分素景,['']


In [7]:
OUTPUT_COLUMSN=['写物', '劝勉', '家庭', '快乐', '悲苦', '政治', '朋友', '游玩']

In [9]:
# Name of the BERT model to use
model_name = 'bert-base-chinese'
# Max length of tokens
max_length = 100
# Load transformers config and set output_hidden_states to False
config = BertConfig.from_pretrained(model_name)
config.output_hidden_states = False
# Load BERT tokenizer
tokenizer = BertTokenizerFast.from_pretrained(pretrained_model_name_or_path = model_name, config = config)

In [10]:
# Tokenize the input (takes some time)
max_length = 100
model_name = 'bert-base-chinese'

x = tokenizer(
    text=poem_without_tag_df['content'].to_list(),
    add_special_tokens=True,
    max_length=max_length,
    truncation=True,
    padding=True, 
    return_tensors='tf',
    return_token_type_ids = False,
    return_attention_mask = False,
    verbose = True)

In [11]:
loaded_model = load_model('multi-label-model')

In [36]:
prediction=loaded_model.predict(x['input_ids'])

In [37]:
prediction['encourage_'][:10]

array([[ 0.57306325, -0.87948287],
       [ 1.2030728 , -1.28399   ],
       [-0.45596835,  0.62340504],
       [ 0.6608906 , -0.72975713],
       [ 1.6606959 , -1.7962083 ],
       [ 0.89311564, -0.6481259 ],
       [ 0.99448   , -0.6496321 ],
       [ 0.94512135, -0.8306388 ],
       [ 1.774136  , -1.7066302 ],
       [ 1.2331916 , -0.90104043]], dtype=float32)

In [39]:
np.argmax(prediction['travel_'],axis=1)

array([1, 1, 0, ..., 0, 0, 0])

In [41]:
#'写物', '劝勉', '家庭', '快乐', '悲苦', '政治', '朋友', '游玩']
poem_without_tag_df['写物']=np.argmax(prediction['object_'],axis=1)
poem_without_tag_df['劝勉']=np.argmax(prediction['encourage_'],axis=1)
poem_without_tag_df['家庭']=np.argmax(prediction['family_'],axis=1)
poem_without_tag_df['快乐']=np.argmax(prediction['happiness_'],axis=1)
poem_without_tag_df['悲苦']=np.argmax(prediction['sadness_'],axis=1)
poem_without_tag_df['政治']=np.argmax(prediction['policy_'],axis=1)
poem_without_tag_df['朋友']=np.argmax(prediction['friend_'],axis=1)
poem_without_tag_df['游玩']=np.argmax(prediction['travel_'],axis=1)

In [48]:
poem_without_tag_df.head()

,content,dynasty,author,tags,star,author_stars,title,tags_list,写物,劝勉,家庭,快乐,悲苦,政治,朋友,游玩
0,一上一上又一上，一上直到高山上。举头红日白云低，四海五湖皆一望。,明代,唐寅,;,1,231,伯虎扮乞儿，作诗骗酒,[''],1,0,1,0,0,1,0,1
1,一个兰舟，双桂桨、顺流东去。但满目、银光万顷，凄其风露。渔火已归鸿雁汊，棹歌更在鸳鸯浦。渐夜...,宋代,汪元量,;,2,57,满江红（吴江秋夜）,[''],0,0,0,0,0,1,0,1
2,一从簪笏事金貂，每借温颜放折腰。长觉身轻离泥滓，忽惊手重捧琼瑶。马因回顾虽增价，桐遇知音已半...,唐代,白居易,;,1,5357,江西裴常侍以优礼见待又蒙赠诗辄叙鄙诚用伸感谢,[''],0,1,0,0,0,1,0,0
3,一以我为牛，一以吾为马。人与之名受不辞，善学庄周者。江海任虚舟，风雨从飘瓦。醉者乘车坠不伤，...,宋代,辛弃疾,;,3,2593,卜算子（用庄语）,[''],0,0,0,0,0,1,0,0
4,一分素景，千家新月，凉露楼台遍洗。宝奁深夜结蛛丝，纴五孔、金针不寐。,宋代,石延年,;,1,6,鹊桥仙·一分素景,[''],0,0,0,1,0,1,0,0


In [44]:
poem_with_tag_df = poem_with_tag_df.drop(['new_tags','new_first_tag'], axis=1)
poem_with_tag_df.head()

,content,dynasty,author,tags,star,author_stars,title,tags_list,写物,劝勉,家庭,快乐,悲苦,政治,朋友,游玩
0,《吴都赋》云：“户藏烟浦，家具画船。”唯吴兴为然。春游之盛，西湖未能过也。己酉岁，予与萧时父...,宋代,姜夔,春游;怀人;,64,279,琵琶仙·《吴都赋》云：「户藏烟浦,"['春游', '怀人']",0,0,0,0,0,1,0,1
1,《廿一史弹词》第三段说秦汉开场词滚滚长江东逝水，浪花淘尽英雄。是非成败转头空。青山依旧在，几...,明代,杨慎,咏史;抒怀;人生;哲理,3244,131,临江仙·滚滚长江东逝水,"['咏史', '抒怀', '人生', '哲理']",1,1,0,0,1,1,0,0
2,《水经》云：“彭蠡之口有石钟山焉。”郦元以为下临深潭，微风鼓浪，水石相搏，声如洪钟。是说也，...,宋代,苏轼,古文观止;纪游;写景;写山,306,4011,石钟山记,"['古文观止', '纪游', '写景', '写山']",1,0,0,0,0,1,0,1
3,【序】辛亥之冬，予载雪诣石湖。止既月，授简索句，且征新声，作此两曲。石湖把玩不已，使工妓隶习...,宋代,姜夔,咏物;梅花,74,279,暗香疏影,"['咏物', '梅花']",0,1,0,0,0,0,0,0
4,〔一枝花〕 攀出墙朵朵花，折临路枝枝柳。花攀红蕊嫩，柳折翠条柔，浪子风流。凭着我折柳攀花手...,元代,关汉卿,;散曲;抒情;生活,140,124,【南吕】一枝花不伏老,"['散曲', '抒情', '生活']",1,0,0,1,1,0,0,0


In [45]:
poem_data = pd.concat([poem_with_tag_df,poem_without_tag_df])

In [46]:
poem_data.head()

,content,dynasty,author,tags,star,author_stars,title,tags_list,写物,劝勉,家庭,快乐,悲苦,政治,朋友,游玩
0,《吴都赋》云：“户藏烟浦，家具画船。”唯吴兴为然。春游之盛，西湖未能过也。己酉岁，予与萧时父...,宋代,姜夔,春游;怀人;,64,279,琵琶仙·《吴都赋》云：「户藏烟浦,"['春游', '怀人']",0,0,0,0,0,1,0,1
1,《廿一史弹词》第三段说秦汉开场词滚滚长江东逝水，浪花淘尽英雄。是非成败转头空。青山依旧在，几...,明代,杨慎,咏史;抒怀;人生;哲理,3244,131,临江仙·滚滚长江东逝水,"['咏史', '抒怀', '人生', '哲理']",1,1,0,0,1,1,0,0
2,《水经》云：“彭蠡之口有石钟山焉。”郦元以为下临深潭，微风鼓浪，水石相搏，声如洪钟。是说也，...,宋代,苏轼,古文观止;纪游;写景;写山,306,4011,石钟山记,"['古文观止', '纪游', '写景', '写山']",1,0,0,0,0,1,0,1
3,【序】辛亥之冬，予载雪诣石湖。止既月，授简索句，且征新声，作此两曲。石湖把玩不已，使工妓隶习...,宋代,姜夔,咏物;梅花,74,279,暗香疏影,"['咏物', '梅花']",0,1,0,0,0,0,0,0
4,〔一枝花〕 攀出墙朵朵花，折临路枝枝柳。花攀红蕊嫩，柳折翠条柔，浪子风流。凭着我折柳攀花手...,元代,关汉卿,;散曲;抒情;生活,140,124,【南吕】一枝花不伏老,"['散曲', '抒情', '生活']",1,0,0,1,1,0,0,0


In [50]:
compression_opts = dict(method='zip', archive_name='poems_with_tags_all.csv') 

poem_data.to_csv('poems_with_tags_all.zip', index=False,compression=compression_opts)  